In [1]:
import pandas as pd
import numpy as np
import random
import math
from pprint import pprint

# import dataset
data = pd.read_csv("agaricus-lepiota.data", header = None)
array = np.array(data)
array = np.array([line for line in array])
random.shuffle(array)
dataset = pd.DataFrame(array)
dataset.columns = ['class', 'cap-shape', 'cap-surface', 'cap-color', 
        'bruises', 'odor', 'gill-attachment', 'gill-spacing', 
        'gill-size', 'gill-color', 'stalk-shape', 'stalk-root', 
        'stalk-surface-above-ring', 'stalk-surface-below-ring', 
        'stalk-color-above-ring', 'stalk-color-below-ring', 'veil-type', 
        'veil-color', 'ring-number', 'ring-type', 'spore-print-color', 
        'population', 'habitat']
# spliting into training dataset and testing dataset, with ratio 0.8
split_index = math.ceil(len(dataset) * 0.8)
training_dataset = dataset.iloc[:split_index].reset_index(drop=True)#We drop the index respectively relabel the index
testing_dataset = dataset.iloc[split_index:].reset_index(drop=True)


In [2]:

def entropy(target_col):
    elements,counts = np.unique(target_col,return_counts = True)
    entropy = np.sum([(-counts[i]/np.sum(counts))*np.log2(counts[i]/np.sum(counts)) for i in range(len(elements))])
    return entropy

def InfoGain(data,split_attribute_name,target_name="class"):   
    #Calculate the entropy of the total dataset
    total_entropy = entropy(data[target_name])

    #Calculate the values and the corresponding counts for the split attribute 
    vals,counts= np.unique(data[split_attribute_name],return_counts=True)
    
    #Calculate the weighted entropy
    Weighted_Entropy = np.sum([(counts[i]/np.sum(counts))*entropy(data.where(data[split_attribute_name]==vals[i]).dropna()[target_name]) for i in range(len(vals))])
    
    #Calculate the information gain
    Information_Gain = total_entropy - Weighted_Entropy
    return Information_Gain


def ID3(data,originaldata,features,target_attribute_name='class',parent_node_class = None, depth = -1, stopping_depth=2):
    #Define the stopping criteria --> If one of this is satisfied, we want to return a leaf node#
    
    #If all target_values have the same value, return this value
    if len(np.unique(data[target_attribute_name])) <= 1 or depth == stopping_depth:
        return np.unique(data[target_attribute_name])[0]
    
    #If the dataset is empty, return the mode target feature value in the original dataset
    elif len(data)==0:
        return np.unique(originaldata[target_attribute_name])[np.argmax(np.unique(originaldata[target_attribute_name],return_counts=True)[1])]
    
    #If the feature space is empty, return the mode target feature value of the direct parent node --> Note that
    #the direct parent node is that node which has called the current run of the ID3 algorithm and hence
    #the mode target feature value is stored in the parent_node_class variable.
    
    elif len(features) ==0:
        return parent_node_class
     
    
    #If none of the above holds true, grow the tree!
    
    else:
        #Set the default value for this node --> The mode target feature value of the current node
        parent_node_class = np.unique(data[target_attribute_name])[np.argmax(np.unique(data[target_attribute_name],return_counts=True)[1])]
        
        #Select the feature which best splits the dataset
        item_values = [InfoGain(data,feature,target_attribute_name) for feature in features] #Return the information gain values for the features in the dataset
        best_feature_index = np.argmax(item_values)
        best_feature = features[best_feature_index]

        #Create the tree structure. The root gets the name of the feature (best_feature) with the maximum information
        #gain in the first run
        tree = {best_feature:{}}
        
        #Remove the feature with the best inforamtion gain from the feature space
        features = [i for i in features if i != best_feature]
        
        #Grow a branch under the root node for each possible value of the root node feature
        depth += 1
        for value in np.unique(data[best_feature]):
            value = value
            #Split the dataset along the value of the feature with the largest information gain and therwith create sub_datasets
            sub_data = data.where(data[best_feature] == value).dropna()
            
            
            #Add the sub tree, grown from the sub_dataset to the tree under the root node
            tree[best_feature][value] = ID3(sub_data,dataset,features,target_attribute_name,parent_node_class, depth, stopping_depth)
            
        return(tree)  

# generating the decision tree, setting the stopping depth 
tree = ID3(training_dataset,training_dataset,training_dataset.columns[1:], depth = -1, stopping_depth=3)

pprint(tree)



{'odor': {'a': 'e',
          'c': 'p',
          'f': 'p',
          'l': 'e',
          'n': {'spore-print-color': {'h': 'e',
                                      'k': 'e',
                                      'n': 'e',
                                      'r': 'p',
                                      'w': {'habitat': {'d': 'p',
                                                        'g': 'e',
                                                        'l': {'cap-shape': {'b': 'p',
                                                                            'f': 'e',
                                                                            'k': 'e',
                                                                            'x': 'e'}},
                                                        'w': 'e'}}}},
          'p': 'p',
          's': 'p',
          'y': 'p'}}


In [3]:
def predict(query,tree,default = 1):
    for key in list(query.keys()):
        if key in list(tree.keys()):
            try:
                result = tree[key][query[key]] 
            except:
                return default
            result = tree[key][query[key]]
            if isinstance(result,dict):
                return predict(query,result)

            else:
                return result
            
            
def test(data,tree):
    #Create new query instances by simply removing the target feature column from the original dataset and 
    #convert it to a dictionary
    queries = data.iloc[:,:-1].to_dict(orient = "records")
    
    #Create a empty DataFrame in whose columns the prediction of the tree are stored
    predicted = pd.DataFrame(columns=["predicted"]) 
    
    #Calculate the prediction accuracy
    for i in range(len(data)):
        predicted.loc[i,"predicted"] = predict(queries[i],tree,1.0) 
    print('The prediction accuracy is: ',(np.sum(predicted["predicted"] == data["class"])/len(data))*100,'%')
    
test(testing_dataset,tree)

The prediction accuracy is:  94.27339901477832 %
